In [1]:
import config
import fasttext

print(fasttext.__VERSION__)

TRAIN_FASTTEXT = True

soynlp=0.0.49
added lovit_textmining_dataset
0.8.3


앞서 Gensim 을 이용한 FastText 를 학습했습니다. Facebook Research 에서도 FastText 코드를 제공합니다. 설치는 pip install 로 가능합니다.

```
pip install fasttext
```

여기에서는 subword enriching 인 unsupervised word embedding 과 document classification 을 위한 supervised word embedding 두 가지 버전을 모두 제공합니다.

## Word representation using subword embedding

Unsupervised FastText 는 띄어쓰기 기준으로 단어가 구분되는 데이터셋이면 됩니다. 한글의 경우 초/중/종성을 구분하면 더 좋은 성능을 얻을 수 있습니다.

```
ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ- ㄴㅗㄹㄹㅏㄴ ㅇㅔ-ㄱㅔ- ㅇㅜ-ㄹㅣ-ㄴㅡㄴ ㄴㅗㄹㄹㅏㄴ ㄷㅏ-
ㅇㅣㄴㅅㅔㅂㅅㅕㄴ ㅈㅓㅇㅁㅏㄹ ㅎㅡㅇㅁㅣ-ㅈㅣㄴㅈㅣㄴㅎㅏ-ㄱㅔ- ㅂㅘㅆㅇㅓㅆㄱㅗ- ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ...
ㄴㅗㄹㄹㅏㄴㅇㅣ-ㅁㅕㄴ ㅁㅜ-ㅈㅗ-ㄱㅓㄴ ㅂㅘ-ㅇㅑ- ㄷㅚㄴㄷㅏ- ㅇㅙ-ㄴㅑ-ㅎㅏ-ㅁㅕㄴ ㅁㅗ-ㄷㅡㄴ ㅈㅏㄱㅍㅜㅁ...
```

In [2]:
from navermovie_comments import get_facebook_fasttext_data

corpus_path = get_facebook_fasttext_data(large=False, supervise=False)

with open(corpus_path, encoding='utf-8') as f:
    for _ in range(3):
        print(next(f).strip()[:50])

ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ- ㄴㅗㄹㄹㅏㄴ ㅇㅔ-ㄱㅔ- ㅇㅜ-ㄹㅣ-ㄴㅡㄴ ㄴㅗㄹㄹㅏㄴ ㄷㅏ-
ㅇㅣㄴㅅㅔㅂㅅㅕㄴ ㅈㅓㅇㅁㅏㄹ ㅎㅡㅇㅁㅣ-ㅈㅣㄴㅈㅣㄴㅎㅏ-ㄱㅔ- ㅂㅘㅆㅇㅓㅆㄱㅗ- ㅋㅡ-ㄹ
ㄴㅗㄹㄹㅏㄴㅇㅣ-ㅁㅕㄴ ㅁㅜ-ㅈㅗ-ㄱㅓㄴ ㅂㅘ-ㅇㅑ- ㄷㅚㄴㄷㅏ- ㅇㅙ-ㄴㅑ-ㅎㅏ-ㅁㅕㄴ 


load_model 을 할 때에는 fasttext_model_name 뒤에 확장자 '.bin' 을 붙여줘야 합니다. .bin 과 .vec 두 가지의 파일을 만들기 때문입니다.

In [3]:
unsupervised_modelname = './fasttext_subword'

if TRAIN_FASTTEXT:
    embedding_model = fasttext.cbow(
        corpus_path,
        unsupervised_modelname,
        minn = 3,
        maxn = 6,
        thread = 8
    )
else:
    embedding_model = fasttext.load_model(
        unsupervised_modelname+'.bin',
        encoding = 'utf-8'
    )

초/중/종성을 나눴기 때문에 cosine similarity 를 계산할 때에도 입력될 단어를 초/중/종성으로 나눠야 합니다. 

In [4]:
from soynlp.hangle import decompose, compose
from soynlp.normalizer import remove_doublespace

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in s.split())

def cosine_similarity(word1, word2):
    word1 = encode(word1)
    word2 = encode(word2)
    return embedding_model.cosine_similarity(word1, word2)

cosine_similarity('재미썼어', '재밌었어')

0.8412253161242274

## Supervised classifier

평점 데이터를 이용하여 classifier 를 학습하는 코드입니다. 8점 이상을 positive, 3점 이하를 negative 라 하였습니다. 

"word_ngrams = 1" parameter는 단어의 ngram 까지도 잡아주는 parameter 입니다만, 지금 공개된 버전에서는 학습이 안되는 버그가 있습니다. (0.8.3) 기본값인 1만을 이용합니다. 

Supervised FastText 는 앞에 label_prefix 를 입력한 형태의 데이터를 가정합니다. 띄어쓰기 기준으로 prefix 가 붙은 단어는 document label 로 이용합니다. 만약 `__label__` 을 prefix 로 이용한다면 아래처럼 구성할 수 있습니다.

```
__label__pos ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ- ㄴㅗㄹㄹㅏㄴ ㅇㅔ-ㄱㅔ- ㅇㅜ-ㄹㅣ-ㄴㅡㄴ ㄴㅗㄹㄹㅏㄴ ㄷㅏ-
__label__pos ㅇㅣㄴㅅㅔㅂㅅㅕㄴ ㅈㅓㅇㅁㅏㄹ ㅎㅡㅇㅁㅣ-ㅈㅣㄴㅈㅣㄴㅎㅏ-ㄱㅔ- ㅂㅘㅆㅇㅓㅆㄱㅗ- ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ...
__label__pos ㄴㅗㄹㄹㅏㄴㅇㅣ-ㅁㅕㄴ ㅁㅜ-ㅈㅗ-ㄱㅓㄴ ㅂㅘ-ㅇㅑ- ㄷㅚㄴㄷㅏ- ㅇㅙ-ㄴㅑ-ㅎㅏ-ㅁㅕㄴ ㅁㅗ-ㄷㅡㄴ ㅈㅏㄱㅍㅜㅁ...
```

이 형태로 데이터를 미리 정리해두었습니다. 평점 기준 1 ~ 3 점은 neg (negative), 8 ~ 10 점은 pos (positive) 영화 평으로 생각합니다. 클래스는 반드시 두 개가 아니어도 괜찮습니다. Supervised FastText 는 Softmax 를 이용하기 때문에 multi class classification 을 지원합니다.

In [5]:
corpus_path = get_facebook_fasttext_data(large=False, supervise=True)

with open(corpus_path, encoding='utf-8') as f:
    for _ in range(3):
        print(next(f).strip()[:50])

__label__pos ㅋㅡ-ㄹㅣ-ㅅㅡ-ㅌㅗ-ㅍㅓ- ㄴㅗㄹㄹㅏㄴ ㅇㅔ-ㄱㅔ- ㅇㅜ-ㄹㅣ-ㄴ
__label__pos ㅇㅣㄴㅅㅔㅂㅅㅕㄴ ㅈㅓㅇㅁㅏㄹ ㅎㅡㅇㅁㅣ-ㅈㅣㄴㅈㅣㄴㅎㅏ-ㄱㅔ- ㅂ
__label__pos ㄴㅗㄹㄹㅏㄴㅇㅣ-ㅁㅕㄴ ㅁㅜ-ㅈㅗ-ㄱㅓㄴ ㅂㅘ-ㅇㅑ- ㄷㅚㄴㄷㅏ- 


In [6]:
supervised_modelname = './fasttext_supervised'

if TRAIN_FASTTEXT:
    classifier_model = fasttext.supervised(
        corpus_path,
        supervised_modelname,
        label_prefix = '__label__',
        word_ngrams = 1,
        thread = 8)
else:
    classifier_model = fasttext.load_model(
        supervised_modelname+'.bin',
        label_prefix = '__label__'
    )

학습된 모델의 labels 를 확인할 수 있습니다.

In [7]:
classifier_model.labels

['pos', 'neg']

학습데이터의 텍스트가 초/중/종성이 분리되어 있기 때문에 입력되는 문장도 동일한 전처리를 거쳐야 합니다.

In [8]:
sent = '언플쩐다 재미없다 이상해'

encode(sent)

'ㅇㅓㄴㅍㅡㄹㅉㅓㄴㄷㅏ- ㅈㅐ-ㅁㅣ-ㅇㅓㅄㄷㅏ- ㅇㅣ-ㅅㅏㅇㅎㅐ-'

classifier.predict() 는 array of str을 입력받아야 합니다. k는 가장 가까운 클래스 k개의 개수입니다. 

classifier 는 각 단어에 대하여 각각 class prediction 을 합니다. 이 결과를 통하여 한 문장, 즉 words 의 classification 까지 하는 함수는 제공되지 않습니다.

In [9]:
words = encode(sent).split()
classifier_model.predict(words,k=2)

[['neg', 'pos'], ['neg', 'pos'], ['neg', 'pos']]

classifier.predict_prob()는 각 단어에 대하여 가까운 k 개 클래스의 확률을 계산합니다. 

In [10]:
classifier_model.predict_proba(words,k=2)

[[('neg', 0.796875), ('pos', 0.201172)],
 [('neg', 0.998047), ('pos', 1.95313e-08)],
 [('neg', 0.996094), ('pos', 0.00195314)]]

문장의 sentiment score 는 각 단어의 score 의 가중합 혹은 평균으로 정의할 수 있습니다.

In [11]:
words = encode('와진짜 개쩐다 영화 졸라 재밌어').split()
classifier_model.predict_proba(words,k=2)

[[('pos', 0.955078), ('neg', 0.0429688)],
 [('pos', 0.992187), ('neg', 0.00585939)],
 [('neg', 0.650391), ('pos', 0.347656)],
 [('neg', 0.515625), ('pos', 0.482422)],
 [('pos', 0.972656), ('neg', 0.0253906)]]

어절 단위의 의미도 학습됩니다. `영화라고` 라는 어절은 거의 부정적인 문맥에서 등장했습니다. `씹노잼`과 같은 단어는 당연히 부정적인 의미이고요.

In [12]:
words = encode('씹노잼 이걸 영화라고 만드냐').split()
classifier_model.predict_proba(words,k=2)

[[('neg', 0.945313), ('pos', 0.0527344)],
 [('neg', 0.990234), ('pos', 0.00781252)],
 [('neg', 0.982422), ('pos', 0.015625)],
 [('neg', 0.998047), ('pos', 1.95313e-08)]]

하지만 `영화`라는 subword 를 포함하는 `영화지` 는 긍정적인 문맥에서 등장하였습니다.

In [13]:
words = encode('이게 영화지').split()
classifier_model.predict_proba(words,k=2)

[[('neg', 0.933594), ('pos', 0.0644531)],
 [('pos', 0.996094), ('neg', 0.00195314)]]

타이포나 띄어쓰기 오류가 포함되어 있더라도 단어의 sentiment 가 판별됩니다.

In [22]:
words = encode('씹ㅏㄹ 이게뭐냐').split()
classifier_model.predict_proba(words,k=2)

[[('neg', 0.998047), ('pos', 1.95313e-08)],
 [('neg', 0.998047), ('pos', 1.95313e-08)]]